# Análise dos Sintomas e Evolução dos Casos
* Quais são os sintomas mais comuns entre os casos notificados de dengue (FEBRE, MIALGIA, CEFALEIA, etc.)?
* Existe uma relação entre a presença de sintomas graves (como LEUCOPENIA ou HEMATOLOG) e a evolução dos casos e/ou óbito?
*	Qual é a prevalência de comorbidades (DIABETES, RENAL, HIPERTENSA, etc.) entre os pacientes diagnosticados com dengue? Qual a relação entre a presença de comorbidades e a evolução/óbito?
* Qual é a taxa de hospitalização (HOSPITALIZ) entre os pacientes diagnosticados com dengue?
* Qual é a taxa de mortalidade (TARGET_OBITO) associada à dengue em 2023?
* Existe alguma tendência temporal entre a data do início dos sintomas (DT_SIN_PRI) e a evolução dos casos (EVOLUCAO) ao longo do tempo?

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("dengue_resumido.csv",sep=';')
df = pd.DataFrame(df)
df.head()

,Unnamed: 0,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
0,0,2023-01-01,120070,12,2023-01-01,2005.0,4017,F,5.0,4.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,1,2023-01-02,120005,12,2023-01-01,1971.0,4051,F,5.0,9.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2,2023-01-02,120005,12,2023-01-01,1995.0,4027,F,9.0,9.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,3,2023-01-02,120030,12,2023-01-01,2002.0,4020,M,6.0,4.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,4,2023-01-02,120005,12,2023-01-02,1990.0,4032,M,6.0,9.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


### Dropando o índice

In [6]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [7]:
df['CS_GESTANT'].unique()

array([ 5.,  9.,  6.,  1.,  4.,  3.,  2., nan])

In [8]:
df['CS_RACA'].unique()

array([ 4.,  9.,  1.,  5.,  2.,  3., nan])

In [9]:
df['CS_ESCOL_N'].unique()

array([nan,  9.,  6.,  8.,  1.,  3., 10.,  4.,  5.,  7.,  2.,  0.])

In [10]:
df.columns

Index(['DT_NOTIFIC', 'ID_MUNICIP', 'SG_UF_NOT', 'DT_SIN_PRI', 'ANO_NASC',
       'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N',
       'HOSPITALIZ', 'EVOLUCAO', 'DT_OBITO', 'FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE'],
      dtype='object')

### Tratamento dos Sintomas

As colunas referentes a sintomas e cormodidades apresentam os seguintes valores:
- 1 = Sim
- 2 = Não

Para melhor entendimento será realizado replace para adequação

In [11]:
colunas = ['FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE']

In [12]:
for valor in colunas:
  df[valor].replace({1:'Sim', 2:'Não'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_24192\495744036.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[valor].replace({1:'Sim', 2:'Não'},inplace=True)


In [13]:
for valor in colunas:
  df[valor].fillna('Não Informado', inplace=True)

In [14]:
df['VOMITO'].value_counts()

VOMITO
Não              1097815
Sim               357884
Não Informado      52954
Name: count, dtype: int64

In [15]:
df[['FEBRE', 'MIALGIA', 'CEFALEIA',
       'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
       'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
       'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
       'ACIDO_PEPT', 'AUTO_IMUNE']]

,FEBRE,MIALGIA,CEFALEIA,EXANTEMA,VOMITO,NAUSEA,DOR_COSTAS,CONJUNTVIT,ARTRITE,ARTRALGIA,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
0,Sim,Não,Sim,Não,Não,Não,Sim,Não,Não,Não,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
1,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
2,Sim,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
3,Sim,Sim,Sim,Não,Sim,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
4,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508648,Sim,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
1508649,Sim,Sim,Sim,Não,Sim,Não,Não,Não,Sim,Sim,...,Não,Não,Sim,Não,Não,Não,Não,Não,Sim,Não
1508650,Sim,Sim,Sim,Não,Não,Não,Sim,Não,Sim,Não,...,Não,Não,Não,Não,Não,Não,Não,Sim,Não,Não
1508651,Sim,Sim,Sim,Não,Sim,Não,Não,Não,Sim,Sim,...,Não,Não,Não,Não,Não,Não,Não,Sim,Não,Não


### Tratamento de Gestante

In [16]:
df['CS_GESTANT'].replace({1:'1º Trimestre',2:'2º Trimestre',3:'3º Trimestre',4:'Idade gestacional ignorada',
                          5:'Não',6:'Não se aplica',9:'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_24192\1575735799.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_GESTANT'].replace({1:'1º Trimestre',2:'2º Trimestre',3:'3º Trimestre',4:'Idade gestacional ignorada',


In [17]:
df['CS_GESTANT'].fillna('Ignorado', inplace=True)

In [18]:
df['CS_GESTANT'].isnull().sum()

0

### Tratamento Raça

In [19]:
df['CS_RACA'].replace({1:'Branca',2:'Preta',3:'Amarela',4:'Parda',
                          5:'Indígena',9:'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_24192\996673323.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_RACA'].replace({1:'Branca',2:'Preta',3:'Amarela',4:'Parda',


In [20]:
df['CS_RACA'].fillna('Ignorado', inplace=True)

In [21]:
df['CS_RACA'].value_counts()

CS_RACA
Branca      660516
Parda       526535
Ignorado    237482
Preta        65251
Amarela      15842
Indígena      3027
Name: count, dtype: int64

In [22]:
df['CS_RACA'].isnull().sum()

0

### Tratamento Escolaridade

In [23]:
df['CS_ESCOL_N'].replace({1:'Fundamental incompleto',2:'Fundamental completo',3:'Fundamental incompleto',
                          4:'Fundamental completo',5:'Ensino Médio incompleto',6:'Ensino Médio completo',
                          7:'Ensino Superior incompleto',8:'Ensino superior completo',9:'Ignorado',10:'Não se aplica', 0: 'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_24192\2399351889.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_ESCOL_N'].replace({1:'Fundamental incompleto',2:'Fundamental completo',3:'Fundamental incompleto',


In [24]:
df['CS_ESCOL_N'].fillna('Ignorado', inplace=True)

In [25]:
df['CS_ESCOL_N'].value_counts()

CS_ESCOL_N
Ignorado                      763293
Ensino Médio completo         249262
Fundamental incompleto        141594
Não se aplica                 100917
Fundamental completo           90212
Ensino Médio incompleto        77064
Ensino superior completo       65083
Ensino Superior incompleto     21228
Name: count, dtype: int64

In [26]:
df['CS_ESCOL_N'].isnull().sum()

0

### Tratamento de Data

In [27]:
import datetime as dt

In [28]:
df[['DT_SIN_PRI','DT_NOTIFIC']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508653 entries, 0 to 1508652
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   DT_SIN_PRI  1508653 non-null  object
 1   DT_NOTIFIC  1508653 non-null  object
dtypes: object(2)
memory usage: 23.0+ MB


In [29]:
df['DT_SIN_PRI'].isnull().sum()

0

In [30]:
df['DT_NOTIFIC'].isnull().sum()

0

In [31]:
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'],format='%Y-%m-%d')
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'],format='%Y-%m-%d')

In [32]:
df.sample(10)

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LEUCOPENIA,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE
183058,2023-02-14,314330,31,2023-02-11,2012.0,4010,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
252266,2023-02-06,411710,41,2023-02-06,1982.0,4040,F,Não,Parda,Ensino Médio completo,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
305164,2023-03-25,330190,33,2023-03-20,1965.0,4058,M,Não se aplica,Ignorado,Ignorado,...,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
1471246,2023-06-08,351050,35,2023-06-03,1956.0,4066,F,Ignorado,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
446232,2023-03-10,353640,35,2023-03-08,1960.0,4063,M,Não se aplica,Parda,Ensino superior completo,...,Não,Não,Não,Sim,Não,Não,Não,Sim,Não,Não
1328970,2023-06-14,311820,31,2023-06-12,1981.0,4041,F,Não,Branca,Ensino Médio completo,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
1495013,2023-06-07,350850,35,2023-06-02,1977.0,4045,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Sim,Não,Não
438186,2023-03-01,354340,35,2023-02-24,1987.0,4035,M,Não se aplica,Branca,Ensino superior completo,...,Não,Não,Sim,Não,Não,Não,Sim,Não,Não,Não
1376053,2023-07-08,410210,41,2023-07-07,1960.0,4062,F,Não,Branca,Ignorado,...,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não
1040638,2023-04-25,350950,35,2023-04-24,1953.0,4069,F,Não,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não


In [33]:
df['DT_OBITO'] = pd.to_datetime(df['DT_OBITO'],format='%Y-%m-%d')

In [34]:
df['DT_OBITO'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1508653 entries, 0 to 1508652
Series name: DT_OBITO
Non-Null Count  Dtype         
--------------  -----         
1531 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 11.5 MB


### Tratamento de Idade

In [35]:
def calcular_idade(valor, ano_nasc):
    # Converte o valor para string para facilitar o acesso aos dígitos
    valor_str = str(valor)
    
    # Verifica o primeiro dígito
    if valor_str[0] == '4':
        # Retorna os últimos três dígitos como idade
        idade = int(valor_str[1:])
    else:
        # Retorna 1 para outros casos
        idade = 1
    
    #Se a idade for maior do que 100, vamos calulá-la a partir da coluna ANO_NASC
    if idade >= 100:
        idade = 2023 - ano_nasc

    return idade

In [36]:
# Aplica a função em cada valor da coluna NU_IDADE_N
df['NU_IDADE_N'] = df.apply(
    lambda row: calcular_idade(row['NU_IDADE_N'], row['ANO_NASC']), axis=1
)

In [37]:
df['NU_IDADE_N'].value_counts()

NU_IDADE_N
23.0     30483
22.0     30012
25.0     29573
26.0     29087
21.0     28895
         ...  
119.0        1
107.0        1
116.0        1
118.0        1
115.0        1
Name: count, Length: 123, dtype: int64

Criando uma coluna categórica para as faixas etárias

In [38]:
faixas_etarias = {
    "80+": (80, 200),
    "75 a 79": (75, 79),
    "70 a 74": (70, 74),
    "65 a 69": (65, 69),
    "60 a 64": (60, 64),
    "55 a 59": (55, 59),
    "50 a 54": (50, 54),
    "45 a 49": (45, 49),
    "40 a 44": (40, 44),
    "35 a 39": (35, 39),
    "30 a 34": (30, 34),
    "25 a 29": (25, 29),
    "20 a 24": (20, 24),
    "15 a 19": (15, 19),
    "10 a 14": (10, 14),
    "5 a 9": (5, 9),
    "0 a 4": (0, 4)
}

In [39]:
#Criando uma coluna categórica para cada faixa etária
def definir_faixa_etaria(idade):
    for faixa, (min_idade, max_idade) in faixas_etarias.items():
        if min_idade <= idade <= max_idade:
            return faixa
    return 'Idade não informada'

In [40]:
df.loc[:, 'Faixa Etária'] = df['NU_IDADE_N'].apply(definir_faixa_etaria)
df.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,F,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,F,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,F,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,M,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34


In [41]:
df_nulo = df[df['NU_IDADE_N'].isnull()]
df_nulo

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
61910,2023-03-26,311340,31,2023-03-25,NaN,NaN,M,Não se aplica,Branca,Ensino Médio completo,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
91510,2023-03-16,311730,31,2023-03-10,NaN,NaN,M,Não se aplica,Branca,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
98405,2023-02-17,310620,31,2023-02-15,NaN,NaN,M,Não se aplica,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Idade não informada
162390,2023-03-29,311940,31,2023-03-27,NaN,NaN,F,Não,Parda,Ensino Médio completo,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
298833,2023-02-11,330220,33,2023-02-08,NaN,NaN,M,Não se aplica,Branca,Fundamental completo,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
300044,2023-02-24,330170,33,2023-02-17,NaN,NaN,F,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
303567,2023-03-18,330170,33,2023-03-15,NaN,NaN,F,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
315532,2023-03-14,110012,11,2023-03-10,NaN,NaN,M,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Idade não informada
327751,2023-03-08,421660,42,2023-03-08,NaN,NaN,M,Não se aplica,Branca,Ensino Médio completo,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Idade não informada
332110,2023-01-31,421660,42,2023-01-30,NaN,NaN,M,Não se aplica,Branca,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Idade não informada


In [42]:
df.dropna(subset = "NU_IDADE_N", inplace=True)

In [43]:
df['NU_IDADE_N'].isnull().sum()

0

### Tratamento Sexo

In [44]:
df['CS_SEXO'].replace({'M':'Masculino', 'F':'Feminino', 'I':'Ignorado'},inplace=True)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_24192\1236526691.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CS_SEXO'].replace({'M':'Masculino', 'F':'Feminino', 'I':'Ignorado'},inplace=True)


In [45]:
df['CS_SEXO'].fillna('Ignorado', inplace=True)

In [46]:
df['CS_SEXO'].isnull().sum()

0

### Verificando Dados Duplicados

In [50]:
df.duplicated().sum()

3177

In [52]:
df.drop_duplicates(inplace=True)

In [53]:
df.duplicated().sum()

0

### Exportando para CSV

In [54]:
df.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,Feminino,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,Feminino,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,Feminino,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,Masculino,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,Masculino,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34


In [55]:
df.to_csv('dengue_resumido_V3.csv',sep=';',index=False)

In [56]:
df2 = pd.read_csv('dengue_resumido_V3.csv',sep=';')
df2.head()

,DT_NOTIFIC,ID_MUNICIP,SG_UF_NOT,DT_SIN_PRI,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,LACO,DOR_RETRO,DIABETES,HEMATOLOG,HEPATOPAT,RENAL,HIPERTENSA,ACIDO_PEPT,AUTO_IMUNE,Faixa Etária
0,2023-01-01,120070,12,2023-01-01,2005.0,17.0,Feminino,Não,Parda,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,15 a 19
1,2023-01-02,120005,12,2023-01-01,1971.0,51.0,Feminino,Não,Ignorado,Ignorado,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,50 a 54
2,2023-01-02,120005,12,2023-01-01,1995.0,27.0,Feminino,Ignorado,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,25 a 29
3,2023-01-02,120030,12,2023-01-01,2002.0,20.0,Masculino,Não se aplica,Parda,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,20 a 24
4,2023-01-02,120005,12,2023-01-02,1990.0,32.0,Masculino,Não se aplica,Ignorado,Ignorado,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,30 a 34
